# AWS CloudFormation Custom Resource
- AWS 리소스 이외의 프로비전 로직을 지원하는 리소스
    - 클라우드포메이션에서 지원하지 않는 리소스 타입 프로비전 혹은 AWS와 무관한 독립적인 로직 수행 가능
- 기본적인 리소스처럼 생성/업데이트/삭제 지원
- 용어
    - Custom Resource Provider : 커스텀 리소스를 프로비전하거나 로직을 수행하는 주체
        - AWS 서비스(예: Lambda, EC2) 혹은 AWS 바깥의 주체도 가능(사람, 온프렘 서버 등)
    - Template Developer : 커스텀 리소스가 포함된 템플릿을 작성하는 사람

## Custom Resource 활용
1. Custom Resource Provider가 주어진 요청에 따라 수행할 로직 및 리소스를 정의
2. 프로바이더가 해당 로직 수행 주체에게 요청을 보낼 수 있는 SNS 토픽 혹은 Lambda를 생성
3. Template Developer가 템플릿 안에 커스텀 리소스를 정의
    - 이 때, 전달할 Input과 SNS ARN 혹은 Lambda ARN을 같이 명시
4. 이후 해당 템플릿을 프로비전(생성, 업데이트, 삭제)할 때 마다 해당 커스텀 리소스에 SNS/Lambda로 요청을 보내고 응답을 대기
5. 해당 리소스가 로직 처리를 완료하면 템플릿에 응답
6. 성공이라면 이후 로직으로 넘어가며 실패(타임아웃, 실패 응답 등)한다면 실패 처리

### Demo - S3 버킷 비우기
- S3의 경우 버킷에 파일이 있으면 삭제 불가능
    - 클라우드포메이션 스택 삭제 시 버킷 삭제 과정에서 문제 발생 가능
- 커스텀 리소스로 미리 버킷의 파일을 정리한 후 삭제를 진행하도록 수정
    - AWS Lambda를 활용한 커스텀 로직 수행
    - 삭제 트리거에만 반응하도록 구현
1. 커스텀 리소스가 없는 스택 생성
2. 프로비전된 버킷에 아무 파일 업로드
3. 이 상태에서 스택 삭제
    - 버킷이 비어있지 않다는 에러 발생
    - 버킷 비우고 다시 삭제
4. 커스텀 리소스가 있는 스택 생성
5. 버킷에 아무 파일이나 업로드
6. 스택 삭제
    - 람다 함수가 S3 객체부터 삭제하는 것을 확인할 수 있다